In [ ]:
import MySQLdb
from pandas.io import sql
import pandas as pd
import numpy as np
conn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="openmrs") 
openmrscursor = conn.cursor()

In [ ]:
df_encounter = pd.read_sql('select encounter_id, visit_id from `encounter` where patient_id=10467237', con = conn) 
df_encounter

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
mimicconn = MySQLdb.connect(host="localhost", user="root", passwd="MYSQLROOTPASSWORD", db="mimic4") 
mimiccursor = mimicconn.cursor()
df_admission_mimic = pd.read_sql('select * from admissions', con = mimicconn) 
df_admission_mimic.head()

In [ ]:
df_admission_mimic_copy = df_admission_mimic[['subject_id', 'hadm_id','admittime']]

In [ ]:
df_admissions_sort = df_admission_mimic_copy.sort_values(by=['admittime'], ascending=False, inplace=False)

In [ ]:
df_admissions_rename = df_admissions_sort.rename({'subject_id': 'patient_id', 'hadm_id':'encounter_id', 'admittime':'encounter_datetime' }, axis='columns')
df_admissions_rename['date_created'] = '2022-01-01'
df_admissions_rename['encounter_type'] = 1

In [ ]:
import uuid
df_admissions_rename['uuid'] = [uuid.uuid4() for x in range(len(df_admissions_rename))]
df_admissions_rename

In [ ]:
encounter_table = df_admissions_rename[['patient_id', 'encounter_id', 'encounter_type', 'encounter_datetime','date_created', 'uuid']]
encounter_table

In [ ]:
import pymysql

# Connect to the database
connection = pymysql.connect(host='localhost',
                         user='root',
                         password='MYSQLROOTPASSWORD',
                         db='openmrs')

# create cursor
cursor=connection.cursor() 

In [ ]:
# creating column list for insertion for person table
cols = "`,`".join([str(i) for i in encounter_table.columns.tolist()])

# Insert DataFrame recrds one by one.
for i,row in encounter_table.iterrows():
    sql1 = "SET FOREIGN_KEY_CHECKS=0"
    sql = "INSERT INTO `encounter` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
    sql2 = "SET FOREIGN_KEY_CHECKS=1"
    try:
        cursor.execute(sql1)
        cursor.execute(sql, tuple(row))
        cursor.execute(sql2)
        
    except MySQLdb.OperationalError:
        continue

    # the connection is not autocommitted by default, so we must commit to save our changes
    connection.commit() 

encounter_location_id = 'UPDATE encounter SET location_id=1 where patient_id!=1'
encounter_creator = 'UPDATE encounter SET creator=1 where patient_id!=1'
encounter_form_id = 'UPDATE encounter SET form_id = NULL' 

cursor.execute(encounter_location_id)
cursor.execute(encounter_creator)
cursor.execute(encounter_form_id)

connection.commit()

# Updating visit_id column in encounter from visit table

In [ ]:
visit_id = 'UPDATE openmrs.encounter oe INNER JOIN openmrs.visit ov ON oe.patient_id = ov.patient_id SET oe.visit_id = ov.visit_id'
cursor.execute(visit_id)

connection.commit() 

In [ ]:
cursor.execute('UPDATE openmrs.encounter SET visit_id=encounter_id where encounter_type=1')
connection.commit()

In [ ]:
df_encounter[df_encounter['patient_id'] == 10215159]